In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict

In [2]:
 # Load the data
data = {
    "eod_balance_training": pd.read_csv(
        "gs://berkabank/production/data/04_processing/eod_balance_training.csv"
    ),
    "incidents": pd.read_csv(
        "gs://berkabank/production/data/03_primary/incidents.csv"
    ),
}

In [6]:
for key in data:
    print(key)
    data[key].to_csv(f"../berkabank/primary/{key}.csv",index=False) 
    print("\n")

eod_balance_training


incidents




In [7]:
data["eod_balance_training"].columns

Index(['account_id', 'balance_date', 'end_of_day_balance', 'daily_amount_flow',
       'n_transactions', 'days_since_account_creation', 'low_balance_streak',
       'district_id'],
      dtype='object')

In [8]:
from dataclasses import dataclass
import pandas as pd
from typing import Dict, List, Any, Callable


@dataclass
class IncidentFeatures:
    incidents: pd.DataFrame
    column_mapping: dict

    def run(self):
        print("----- Running IncidentFeatures...")
        incident_features = pd.DataFrame()
        print("----- IncidentFeatures completed.")
        return incident_features


@dataclass
class EODBFeatures:
    eod_balance_training: pd.DataFrame
    column_mapping: dict
    aggregations:dict

    def run(self):
        print("----- Running EODBFeatures...")
        eodb_features_output = pd.DataFrame()
        print("----- EODBFeatures completed.")
        return eodb_features_output


@dataclass
class PrimaryFeatures:
    incident_features: IncidentFeatures
    eodb_features: EODBFeatures

    def __post_init__(self):
        print("--- Initializing PrimaryFeatures...")
        self.incident_features_output = self.incident_features.run()
        self.eodb_features_output = self.eodb_features.run()
        print("--- PrimaryFeatures initialized.")

    def run(self):
        print("--- Running PrimaryFeatures...")
        primary_features_output = pd.DataFrame()
        print("--- PrimaryFeatures completed.")
        return primary_features_output


@dataclass
class DerivedFeatures:
    primary_features: PrimaryFeatures

    def __post_init__(self):
        print("--- Initializing DerivedFeatures...")
        self.primary_features_output = self.primary_features.run()
        print("--- DerivedFeatures initialized.")

    def run(self):
        print("--- Running DerivedFeatures...")
        derived_features = pd.DataFrame()
        print("--- DerivedFeatures completed.")
        return derived_features


@dataclass
class FeatureEngineering:
    primary_features: PrimaryFeatures
    derived_features: DerivedFeatures

    def run(self):
        print("Running FeatureEngineering...")
        self.derived_features.run()
        print("FeatureEngineering completed.")

In [ ]:

@dataclass
class EODBFeatures:
    eod_balance_training: pd.DataFrame
    column_mapping: dict
    aggregations:dict

    def run(self):
        print("----- Running EODBFeatures...")
        eodb_features_output = pd.DataFrame()
        
        print("----- EODBFeatures completed.")
        return eodb_features_output


In [9]:
column_mapping = {
    "incidents":{
        "account_id": "account_id",
        "incident_date": "incident_date",
        "district_id": "district_id",
        "t0": "t0",
        "t1": "t1",
    },
    "eod_balance_training":{
        "account_id": "account_id",
        "balance_date": "balance_date",
        "end_of_day_balance": "end_of_day_balance",
        "daily_amount_flow": "daily_amount_flow",
        "n_transactions": "n_transactions",
        "days_since_account_creation": "days_since_account_creation",
        "low_balance_streak": "low_balance_streak",
        "district_id": "district_id",
    }
    
}

aggregations = {
    "time_periods_days":[str(i) for i in range(1, 31)],
    "functions" : ["mean", "std", "min", "max", "sum"],
    "columns": ["end_of_day_balance" , "daily_amount_flow", "n_transactions"]
}

In [125]:
df = data["eod_balance_training"].loc[:,["account_id","balance_date","end_of_day_balance" , "daily_amount_flow", "n_transactions"]].copy()

In [126]:
df.head()

,account_id,balance_date,end_of_day_balance,daily_amount_flow,n_transactions
0,576,1993-01-11,7107.0,6207.0,1.0
1,576,1993-01-12,7107.0,0.0,1.0
2,576,1993-01-13,7107.0,0.0,1.0
3,576,1993-01-14,7107.0,0.0,1.0
4,576,1993-01-15,7107.0,0.0,1.0


In [127]:
# Configure each class with specific parameters
incident_features = IncidentFeatures(data["incidents"], column_mapping["incidents"])
eodb_features = EODBFeatures(
    data["eod_balance_training"], column_mapping["eod_balance_training"],aggregations
)
primary_features = PrimaryFeatures(
   
    incident_features,
    eodb_features,
)
derived_features = DerivedFeatures(primary_features)

# Run the FeatureEngineering pipeline
feature_engineering = FeatureEngineering(primary_features, derived_features)
features = feature_engineering.run()

--- Initializing PrimaryFeatures...
----- Running IncidentFeatures...
----- IncidentFeatures completed.
----- Running EODBFeatures...
----- EODBFeatures completed.
--- PrimaryFeatures initialized.
--- Initializing DerivedFeatures...
--- Running PrimaryFeatures...
--- PrimaryFeatures completed.
--- DerivedFeatures initialized.
Running FeatureEngineering...
--- Running DerivedFeatures...
--- DerivedFeatures completed.
FeatureEngineering completed.


In [128]:
df.shape

(688119, 5)

In [196]:
df = data["eod_balance_training"].loc[:,["account_id","balance_date","end_of_day_balance" , "daily_amount_flow", "n_transactions"]].copy()

In [175]:


@dataclass
class EODBFeatures:
    eod_balance_training: pd.DataFrame
    column_mapping: dict
    aggregations: List[str]

    def run(self):
        print("----- Running EODBFeatures...")
        eodb_features_output = pd.DataFrame()
        print("----- EODBFeatures completed.")
        return eodb_features_output
    


# Create independent rolling statistics for each account by time period
max_t = 20
min_t = 5
time_periods_days = [i for i in range(min_t, max_t+1)]
agg_funcs = ["mean", "max", "sum" , "std", "median" ]
category = ['inflow', 'outflow']
feature = 'daily_amount_flow'

aggregations = []
df['flow_category'] = (df['daily_amount_flow'] > 0).astype(str).replace({'True': 'inflow', 'False': 'outflow'})
for time_period in time_periods_days:
    for func in agg_funcs:
        for cat in category:
            naming_f = f'f_{feature}__{cat}_rolling_{func}_{time_period}_days'
            df[naming_f] = df.loc[df['flow_category'] == cat,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)
            if cat == 'outflow':
                df[naming_f] = df[naming_f].abs()
            aggregations.append(naming_f)

/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/1905528839.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[naming_f] = df.loc[df['flow_category'] == cat,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)
/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/1905528839.py:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[naming_f] = df.loc[df['flow_category'] == cat,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(leve

In [183]:
feature_name = aggregations[0]

In [218]:
def create_aggregations(max_t:int = 20, min_t:int = 5, agg_funcs:List[str] = ["mean", "max", "sum" , "std", "median"], category:List[str] = ['inflow', 'outflow'], features:List[str] = ['daily_amount_flow']):
    time_periods_days = [i for i in range(min_t, max_t+1)]
    aggregations = []
    for feature in features:
        for time_period in time_periods_days:
            for func in agg_funcs:
                for cat in category:
                    naming_f = f'f_{feature}__{cat}_rolling_{func}_{time_period}_days'
                    aggregations.append(naming_f)
    return aggregations

In [219]:
def make_aggregations(df:pd.DataFrame, agg_funcs:List[str]):
    df['flow_category'] = (df['daily_amount_flow'] > 0).astype(str).replace({'True': 'inflow', 'False': 'outflow'})
    for aggregation in agg_funcs:
        parts = aggregation.split('__')
        feature = parts[0][2:]
        category = parts[1].split('_rolling_')[0]
        func = parts[1].split('_rolling_')[1].split('_')[0]
        time_period = int(parts[1].split('_rolling_')[1].split('_')[1])
        df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)
        if category == 'outflow':
            df[aggregation] = df[aggregation].abs()
    return df

In [216]:
make_aggregations(df, aggregations)

f_daily_amount_flow__inflow_rolling_mean_5_days
daily_amount_flow inflow mean 5
f_daily_amount_flow__outflow_rolling_mean_5_days
daily_amount_flow outflow mean 5
f_daily_amount_flow__inflow_rolling_max_5_days
daily_amount_flow inflow max 5
f_daily_amount_flow__outflow_rolling_max_5_days
daily_amount_flow outflow max 5
f_daily_amount_flow__inflow_rolling_sum_5_days
daily_amount_flow inflow sum 5
f_daily_amount_flow__outflow_rolling_sum_5_days
daily_amount_flow outflow sum 5
f_daily_amount_flow__inflow_rolling_std_5_days
daily_amount_flow inflow std 5
f_daily_amount_flow__outflow_rolling_std_5_days
daily_amount_flow outflow std 5
f_daily_amount_flow__inflow_rolling_median_5_days
daily_amount_flow inflow median 5
f_daily_amount_flow__outflow_rolling_median_5_days
daily_amount_flow outflow median 5
f_daily_amount_flow__inflow_rolling_mean_6_days
daily_amount_flow inflow mean 6
f_daily_amount_flow__outflow_rolling_mean_6_days
daily_amount_flow outflow mean 6
f_daily_amount_flow__inflow_roll

/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)
/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).re

f_daily_amount_flow__inflow_rolling_median_14_days
daily_amount_flow inflow median 14
f_daily_amount_flow__outflow_rolling_median_14_days
daily_amount_flow outflow median 14


/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)
/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).re

f_daily_amount_flow__inflow_rolling_mean_15_days
daily_amount_flow inflow mean 15
f_daily_amount_flow__outflow_rolling_mean_15_days
daily_amount_flow outflow mean 15


...

f_daily_amount_flow__inflow_rolling_median_20_days
daily_amount_flow inflow median 20


/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)


f_daily_amount_flow__outflow_rolling_median_20_days
daily_amount_flow outflow median 20


/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_73300/802122447.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[aggregation] = df.loc[df['flow_category'] == category,:].groupby('account_id')[feature].rolling(time_period).agg(func).reset_index(level=0, drop=True)


,account_id,balance_date,end_of_day_balance,daily_amount_flow,n_transactions,flow_category,f_daily_amount_flow__inflow_rolling_mean_5_days,f_daily_amount_flow__outflow_rolling_mean_5_days,f_daily_amount_flow__inflow_rolling_max_5_days,f_daily_amount_flow__outflow_rolling_max_5_days,...,f_daily_amount_flow__inflow_rolling_mean_20_days,f_daily_amount_flow__outflow_rolling_mean_20_days,f_daily_amount_flow__inflow_rolling_max_20_days,f_daily_amount_flow__outflow_rolling_max_20_days,f_daily_amount_flow__inflow_rolling_sum_20_days,f_daily_amount_flow__outflow_rolling_sum_20_days,f_daily_amount_flow__inflow_rolling_std_20_days,f_daily_amount_flow__outflow_rolling_std_20_days,f_daily_amount_flow__inflow_rolling_median_20_days,f_daily_amount_flow__outflow_rolling_median_20_days
0,576,1993-01-11,7107.0,6207.0,1.0,inflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,576,1993-01-12,7107.0,0.0,1.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,576,1993-01-13,7107.0,0.0,1.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,576,1993-01-14,7107.0,0.0,1.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,576,1993-01-15,7107.0,0.0,1.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688114,4375,1996-11-10,800.0,0.0,0.0,outflow,NaN,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688115,880,1996-12-08,700.0,700.0,0.0,inflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688116,880,1996-12-09,700.0,0.0,0.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688117,880,1996-12-10,700.0,0.0,0.0,outflow,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
